In [6]:
from PIL import Image
import numpy
from os import listdir
import pandas
from sklearn.metrics import confusion_matrix

pathLabel = "C:/Users/user/Desktop/Python/Mole/Data/Label/label.csv"  ##  Label路徑
imageLabelTable = pandas.read_csv(pathLabel)
imageLabelTable

pathImage = "C:/Users/user/Desktop/Python/Mole/Data/Image/"  ##  圖檔路徑

## 圖的資料
ImageData = []
ImageLabel = []
for index, name in enumerate(listdir(pathImage)):
    
    ##  Load圖
    image = Image.open(pathImage + name)
    image = image.resize( (32, 32), Image.BILINEAR )
    
    ##  圖的array
    ImageData.append(numpy.array( image ))

    ##  圖的label
    iLabel = imageLabelTable[imageLabelTable.Image == name].Label.item()
    ImageLabel.append(iLabel)
    
ImageData = numpy.array(ImageData)  ##  print(ImageData.shape)  ##  幹!!這麼漂亮!! => 直接整理好!!
ImageLabel = numpy.array(ImageLabel)
ImageLabel = ImageLabel.reshape(ImageLabel.size, 1)

In [7]:
from keras import preprocessing
generator = preprocessing.image.ImageDataGenerator(rotation_range=30)
batch = 20  ##  每組 K 個圖片
dataGenerator = generator.flow(ImageData, ImageLabel, batch_size=batch)

K = 40  ##  產生 K 組
for i in range(K):
    tempX, tempY = next(dataGenerator)
    ImageData = numpy.concatenate((ImageData, tempX), axis=0)
    ImageLabel = numpy.concatenate((ImageLabel, tempY), axis=0)

In [8]:
Data = (ImageData, ImageLabel)

In [14]:
from keras import utils
from sklearn import model_selection
import keras
from sklearn.metrics import accuracy_score
confuseMatrixinFold = []
accuracyinFold = []
K = 3
i = 1
StratifiedKFold = model_selection.StratifiedKFold(n_splits=K, random_state=0, shuffle=False)
for trainIndex, validIndex in StratifiedKFold.split(Data[0], Data[1]):
    if i == 1:
        print("Here are ", K, "-folds cross validation loop", "\n\n")
        
    print("Enter ", i, "fold")
    
    ## Train X in k-fold
    ## Valid X in k-fold
    trainXinFold = Data[0][trainIndex, :, :, :]   
    validXinFold = Data[0][validIndex, :, :, :]   
    trainYinFold = Data[1][trainIndex]
    validYinFold = Data[1][validIndex]
    
    trainXinFold = trainXinFold.astype('float32')
    validXinFold = validXinFold.astype('float32')
    trainXinFold /= 255
    validXinFold /= 255
    
    trainYinFold = utils.to_categorical(trainYinFold, 10)
    validYinFold = utils.to_categorical(validYinFold, 10)
    
    model = keras.models.Sequential()
    model.add(keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu", input_shape=(32, 32, 3)))
    model.add(keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(keras.layers.Dropout(0.25))
    model.add(keras.layers.Flatten())  ##  拉直 
    model.add(keras.layers.Dense(128, activation='relu'))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(10, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    
    model.fit(trainXinFold, trainYinFold, batch_size=20, epochs=10, 
              verbose=1, validation_data=(validXinFold, validYinFold))
    
    validYhatinFold = model.predict(validXinFold)
    confuseMatrixinFold.append( confusion_matrix(validYinFold.argmax(axis=-1), validYhatinFold.argmax(axis=-1)) )
    accuracyinFold.append( accuracy_score(validYinFold.argmax(axis=-1), validYhatinFold.argmax(axis=-1)) )
    i = i + 1

Here are  3 -folds cross validation loop 


Enter  1 fold
Train on 775 samples, validate on 393 samples
Epoch 1/10
775/775 [==============================] - 3s 4ms/step - loss: 2.3271 - acc: 0.1135 - val_loss: 2.2821 - val_acc: 0.1145
Epoch 2/10
775/775 [==============================] - 2s 3ms/step - loss: 2.2850 - acc: 0.1510 - val_loss: 2.2365 - val_acc: 0.2392
Epoch 3/10
775/775 [==============================] - 2s 3ms/step - loss: 2.2148 - acc: 0.2323 - val_loss: 2.1077 - val_acc: 0.2901
Epoch 4/10
775/775 [==============================] - 2s 3ms/step - loss: 2.0168 - acc: 0.2903 - val_loss: 1.8396 - val_acc: 0.3435
Epoch 5/10
775/775 [==============================] - 2s 3ms/step - loss: 1.8414 - acc: 0.3613 - val_loss: 1.6353 - val_acc: 0.4173
Epoch 6/10
775/775 [==============================] - 2s 3ms/step - loss: 1.6220 - acc: 0.4439 - val_loss: 1.4229 - val_acc: 0.5267
Epoch 7/10
775/775 [==============================] - 2s 3ms/step - loss: 1.3904 - acc: 0.5239 - val_los

In [15]:
accuracyinFold

[0.6335877862595419, 0.6211340206185567, 0.7054263565891473]